# Introduction
In this notebook, we perform the cleaning of the player statistics data.
At the beginning, we present a brief pipeline summarizing the core cleaning process in three code blocks for a quick overview.

Further below, you will find a more detailed and structured approach, including intermediate steps and explanations to ensure a clear understanding of the data cleaning decisions made.

The goal is to create a reliable and well-prepared dataset that can be used for subsequent analysis steps.

## Setup

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import yaml
from IPython.display import Markdown, display
import zipfile

In [5]:
# load yaml
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
    DATA_PATH = config.get("data_path")
    if DATA_PATH is None:
        print("ERROR: No data path provided")
    USE_DRIVE = bool(config.get("use_drive", False))

## Cleaning Pipeline

In [9]:
# --- Block 1: Load and clean raw player statistics data ---

# Load the raw CSV file from the ZIP archive
zip_path = os.path.join(DATA_PATH, "NBA_Boxscores.zip")
csv_filename = "PlayerStatistics.csv"

with zipfile.ZipFile(zip_path) as archive:
    with archive.open(csv_filename) as file:
        player_statistics_df = pd.read_csv(file)

# Remove rows with negative or missing minutes played (considered inactive)
player_statistics_df = player_statistics_df[player_statistics_df['numMinutes'].fillna(0) >= 0]
player_statistics_df = player_statistics_df[player_statistics_df['numMinutes'] > 0]

# Fix swapped field goals
mask_fg_swapped = player_statistics_df['fieldGoalsMade'] > player_statistics_df['fieldGoalsAttempted']
player_statistics_df.loc[mask_fg_swapped, ['fieldGoalsMade', 'fieldGoalsAttempted']] = (
    player_statistics_df.loc[mask_fg_swapped, ['fieldGoalsAttempted', 'fieldGoalsMade']].values
)

# Fix swapped free throws
mask_ft_swapped = player_statistics_df['freeThrowsMade'] > player_statistics_df['freeThrowsAttempted']
player_statistics_df.loc[mask_ft_swapped, ['freeThrowsMade', 'freeThrowsAttempted']] = (
    player_statistics_df.loc[mask_ft_swapped, ['freeThrowsAttempted', 'freeThrowsMade']].values
)

# Set percentage = 0 if attempted == 0
player_statistics_df['fieldGoalsPercentage'] = np.where(
    player_statistics_df['fieldGoalsAttempted'] > 0,
    player_statistics_df['fieldGoalsMade'] / player_statistics_df['fieldGoalsAttempted'],
    0.0  # instead of np.nan
)

player_statistics_df['freeThrowsPercentage'] = np.where(
    player_statistics_df['freeThrowsAttempted'] > 0,
    player_statistics_df['freeThrowsMade'] / player_statistics_df['freeThrowsAttempted'],
    0.0
)

# Remove entries with personal fouls > 6 (not possible in an NBA game)
player_statistics_df = player_statistics_df[player_statistics_df['foulsPersonal'] <= 6]

# Convert gameDate to datetime format for further filtering/sorting
player_statistics_df['gameDate'] = pd.to_datetime(player_statistics_df['gameDate'])

player_statistics_df.head()

C:\Users\chris\AppData\Local\Temp\ipykernel_17180\297795718.py:9: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  player_statistics_df = pd.read_csv(file)


,firstName,lastName,personId,gameId,gameDate,playerteamCity,playerteamName,opponentteamCity,opponentteamName,gameType,...,threePointersPercentage,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints
0,Kevin,Durant,201142,22401058,2025-03-26 22:00:00,Phoenix,Suns,Boston,Celtics,Regular Season,...,0.750,5.0,5.0,1.0,1.0,0.0,1.0,2.0,1.0,-22.0
1,Al,Horford,201143,22401058,2025-03-26 22:00:00,Boston,Celtics,Phoenix,Suns,Regular Season,...,0.286,4.0,4.0,1.0,6.0,4.0,10.0,1.0,0.0,24.0
2,Jrue,Holiday,201950,22401058,2025-03-26 22:00:00,Boston,Celtics,Phoenix,Suns,Regular Season,...,0.400,2.0,2.0,1.0,2.0,0.0,2.0,1.0,0.0,28.0
3,Mason,Plumlee,203486,22401058,2025-03-26 22:00:00,Phoenix,Suns,Boston,Celtics,Regular Season,...,0.000,0.0,0.0,0.0,4.0,1.0,5.0,2.0,1.0,1.0
4,Vasilije,Micic,203995,22401058,2025-03-26 22:00:00,Phoenix,Suns,Boston,Celtics,Regular Season,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,-3.0


In [10]:
# --- Block 2: Define valid regular season games (including Emirates Cup, excluding IST Finals) ---

# Select all official Regular Season games
regular_season_df = player_statistics_df[player_statistics_df['gameType'] == 'Regular Season']

# Include NBA Emirates Cup games, excluding the Championship Final (Dec 17, 2024)
emirates_cup_non_final_df = player_statistics_df[
    (player_statistics_df['gameType'] == 'NBA Emirates Cup') &
    (player_statistics_df['gameSubLabel'] != 'Championship')
]

# Combine Regular Season and Emirates Cup games into one dataset
combined_df = pd.concat([regular_season_df, emirates_cup_non_final_df])

# Final cleanup: remove any remaining Championship-labeled games (e.g. misclassified 2024 IST Final)
regular_season_player_statistics_df = combined_df[
    combined_df['gameSubLabel'] != 'Championship'
].reset_index(drop=True)

In [11]:
# --- Block 3: Final cleanup – drop unused columns & sort dataset ---

# Drop columns that are no longer needed for analysis
# - 'gameLabel': promotional game titles (e.g. "Paris Game")
# - 'gameSubLabel': In-Season Tournament structure (already filtered)
# - 'seriesGameNumber': always 0.0 in regular season
cols_to_drop = ['gameLabel', 'gameSubLabel', 'seriesGameNumber']
regular_season_player_statistics_df = regular_season_player_statistics_df.drop(columns=cols_to_drop)

# Sort the dataset by game date (most recent games first)
regular_season_player_statistics_df = regular_season_player_statistics_df.sort_values(
    by='gameDate', ascending=False
).reset_index(drop=True)

In [13]:
# Save cleaned dataset
output_path = os.path.join(DATA_PATH, "player_statistics_regular_season.csv")

# Ensure the directory exists
os.makedirs(DATA_PATH, exist_ok=True)

regular_season_player_statistics_df.to_csv(output_path, index=False)
print(f"File saved successfully to {output_path}")

File saved successfully to C:/Users/chris/Documents/nba_data/data\player_statistics_regular_season.csv


In the following sections, we present a detailed walkthrough of the player statistics data cleaning process.

Each step is carefully explained and justified to ensure a transparent and reproducible preparation of the dataset for further analysis.

The process includes data corrections, filtering of relevant games, and final structuring of the cleaned dataset.

In [14]:
# Load the raw CSV file from the ZIP archive
zip_path = os.path.join(DATA_PATH, "NBA_Boxscores.zip")
csv_filename = "PlayerStatistics.csv"

with zipfile.ZipFile(zip_path) as archive:
    with archive.open(csv_filename) as file:
        player_statistics_df = pd.read_csv(file)

rows_before_cleaning = len(player_statistics_df)
print(f"Rows before cleaning:   {rows_before_cleaning:,}")

C:\Users\chris\AppData\Local\Temp\ipykernel_17180\3928930824.py:7: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  player_statistics_df = pd.read_csv(file)


Rows before cleaning:   1,621,084


In [15]:
# Detailed column overview with dtype, value range/examples, and missing values
print("Detailed column overview:\n")
for col in player_statistics_df.columns:
    dtype = player_statistics_df[col].dtype
    missing = player_statistics_df[col].isnull().sum()

    if dtype == 'object':
        example_values = player_statistics_df[col].dropna().unique()[:3]  # Show a few example values
        # print in an output has left justfied columns, each of 100 characters width
        examples = f"Examples: {str(list(example_values))}"
        print(f"| {str(col):<30} | {str(dtype):>10} | {examples:<80} | Missing: {missing:>10} |")
    else:
        min_val = player_statistics_df[col].min()
        max_val = player_statistics_df[col].max()
        r = f"Range: {min_val} to {max_val}"
        print(f"| {str(col):<30} | {str(dtype):>10} | {r:<80} | Missing: {missing:>10} |")

Detailed column overview:

| firstName                      |     object | Examples: ['Kevin', 'Al', 'Jrue']                                                | Missing:          0 |
| lastName                       |     object | Examples: ['Durant', 'Horford', 'Holiday']                                       | Missing:          0 |
| personId                       |      int64 | Range: 2 to 1962937827                                                           | Missing:          0 |
| gameId                         |      int64 | Range: 10300001 to 62400001                                                      | Missing:          0 |
| gameDate                       |     object | Examples: ['2025-03-26 22:00:00', '2025-03-26 21:00:00', '2025-03-26 19:30:00']  | Missing:          0 |
| playerteamCity                 |     object | Examples: ['Phoenix', 'Boston', 'Denver']                                        | Missing:          0 |
| playerteamName                 |     object | Example

**firstName, lastName** First and last name of the player

**personId**	Player's unique ID

**gameId**	Unique ID of the game, needed for joins

**gameDate**	Date and time of the game
Imported as object -> better to convert to datetime

**playerteamCity**  Indicates the city of the team the player played for in that game (e.g., Los Angeles → Lakers)

**opponentteamCity**  City of the opposing team (e.g., San Francisco → Warriors)

**gameType**	Type of game (Regular Season, Playoffs, Preseason), can be useful for segmentation

**gameLabel** An event- or competition-related label used to mark special games or tournaments. Examples:

"NBA Paris Games" → International game in Paris

"Emirates NBA Cup" → Game within the new In-Season Tournament

"NBA Mexico City Game" → Game in Mexico City

**gameSubLabel**	An additional label – used within playoffs or tournaments, e.g.:

"Championship" → Final game

"West Semifinal" → Western Conference Semifinals

"East Semifinal" → Eastern Conference Semifinals

There are many missing values for gameLabel and gameSubLabel, as these labels are only applied to “special” games → can be relevant for specific research questions focusing on those games

**seriesGameNumber**	Indicates which game number it is within a specific playoff series (Best-of-7 format).
Series only exist during the playoffs → In the regular season, this value is NaN

**win** 1 = Player’s team won, 0 = lost, classification target

**home**	1 = Home game, 0 = Away game, relevant for home-court advantage analysis

**numMinutes**	Player’s minutes played, negative values -> errors (needs cleaning)

**points**	Total points scored by the player in the specific game

**assists**	Number of assists (passes leading to a score)

**blocks**	Number of opponent shots blocked

**steals**	Number of times the player stole the ball from the opponent

**fieldGoalsAttempted**	Field goal attempts (2-pointers and 3-pointers) –> all shot attempts excluding free throws

**fieldGoalsMade**	Field goals made

**fieldGoalsPercentage**	Field goal shooting percentage (normally 0–1 – here values up to 9?) → possibly wrongly scaled –> needs verification

**threePointersAttempted, Made, Percentage**	Attempts, makes, and shooting percentage for 3-point shots Percentage looks plausible (0–1 range)

**freeThrowsAttempted, Made, Percentage**	Free throws - attempts, makes, percentage. freeThrowsPercentage goes up to 11 –> should be 0–1 → probably scaling error

**reboundsDefensive, reboundsOffensive, reboundsTotal**
Rebounds split into defensive, offensive, and total

Defensive rebounds = after missed opponent shots

Offensive rebounds = after missed own shots

reboundsTotal = sum of both

**foulsPersonal** Personal fouls committed by the player in the game
NBA rules allow max. 6 fouls per player before disqualification
→ Values up to 23 in the dataset likely indicate errors → should be checked

**turnovers** Number of turnovers (e.g., bad passes, traveling, opponent steals)
Important metric for evaluating efficiency and decision-making – high turnover count = generally negative

**plusMinusPoints** Point differential while the player was on the court: team score minus opponent score
Example: Player plays 25 minutes, during which his team scores 60 and opponent 50 → plusMinusPoints = +10
Positive values = positive impact, even without scoring many points

In [16]:
# 1. Minutes less than 0
minutes_neg = player_statistics_df[player_statistics_df['numMinutes'] < 0]

# 2. Field goal and free throw percentages greater than 1
fg_perc_invalid = player_statistics_df[player_statistics_df['fieldGoalsPercentage'] > 1]
ft_perc_invalid = player_statistics_df[player_statistics_df['freeThrowsPercentage'] > 1]

# 3. Personal fouls greater than 6
fouls_invalid = player_statistics_df[player_statistics_df['foulsPersonal'] > 6]

# 4. Rows with many NaNs in key stat columns
stat_cols = ['points', 'assists', 'blocks', 'steals', 'reboundsTotal']
mostly_nan_rows = player_statistics_df[player_statistics_df[stat_cols].isnull().sum(axis=1) >= 3]

print(f"Negative minutes: {len(minutes_neg)}")
print(f"FG% > 1: {len(fg_perc_invalid)}")
print(f"FT% > 1: {len(ft_perc_invalid)}")
print(f"Fouls > 6: {len(fouls_invalid)}")
print(f"Stat rows with >=3 NaNs: {len(mostly_nan_rows)}")

Negative minutes: 3
FG% > 1: 25
FT% > 1: 71
Fouls > 6: 42
Stat rows with >=3 NaNs: 1219


In [17]:
# Filter rows where field goal or free throw percentages are suspiciously high (> 1.0)
suspicious_fg = player_statistics_df[player_statistics_df['fieldGoalsPercentage'] > 1.0]
suspicious_ft = player_statistics_df[player_statistics_df['freeThrowsPercentage'] > 1.0]

# Columns we want to inspect for checking if the percentages might be inverted
columns_to_show = [
    'fieldGoalsMade', 'fieldGoalsAttempted', 'fieldGoalsPercentage',
    'freeThrowsMade', 'freeThrowsAttempted', 'freeThrowsPercentage'
]

# Display a few suspicious field goal rows
print("Suspicious Field Goal Percentage values (> 1.0):\n")
display(suspicious_fg[columns_to_show].head(10))

# Display a few suspicious free throw rows
print("\nSuspicious Free Throw Percentage values (> 1.0):\n")
display(suspicious_ft[columns_to_show].head(10))

Suspicious Field Goal Percentage values (> 1.0):



,fieldGoalsMade,fieldGoalsAttempted,fieldGoalsPercentage,freeThrowsMade,freeThrowsAttempted,freeThrowsPercentage
1513709,8.0,7.0,1.143,0.0,0.0,0.000
1515863,2.0,1.0,2.000,1.0,1.0,1.000
1519539,3.0,2.0,1.500,0.0,2.0,0.000
1521313,2.0,1.0,2.000,1.0,1.0,1.000
1534431,15.0,7.0,2.142,5.0,7.0,0.713
1534609,11.0,9.0,1.221,6.0,6.0,1.000
1538890,11.0,2.0,5.500,4.0,6.0,0.667
1557016,9.0,1.0,9.000,2.0,2.0,1.000
1561895,7.0,2.0,3.500,6.0,7.0,0.856
1561940,3.0,2.0,1.500,1.0,2.0,0.500



Suspicious Free Throw Percentage values (> 1.0):



,fieldGoalsMade,fieldGoalsAttempted,fieldGoalsPercentage,freeThrowsMade,freeThrowsAttempted,freeThrowsPercentage
1537806,4.0,16.0,0.250,8.0,3.0,2.666
1538981,11.0,25.0,0.440,5.0,1.0,5.000
1540400,0.0,5.0,0.000,21.0,2.0,10.500
1541506,0.0,10.0,0.000,7.0,1.0,7.000
1544001,3.0,0.0,0.000,2.0,1.0,2.000
1544003,10.0,0.0,0.000,6.0,5.0,1.199
1561439,2.0,10.0,0.200,4.0,2.0,2.000
1561441,1.0,4.0,0.250,5.0,3.0,1.667
1561442,2.0,6.0,0.333,3.0,2.0,1.500
1561447,11.0,25.0,0.440,11.0,9.0,1.221


"made" shots are greater than "attempted" shots, which is not possible. This suggests that the values for "made" and "attempted" were accidentally swapped.

The 1219 NaNs in the stat columns correspond to players who were listed but did not play a single second → inactive → will be removed in the following step.

In [18]:
# 1. Remove rows with negative minutes
player_statistics_df = player_statistics_df[player_statistics_df['numMinutes'].fillna(0) >= 0]

# 2. Personal fouls greater than 6 → remove (not allowed in the NBA)
player_statistics_df = player_statistics_df[player_statistics_df['foulsPersonal'] <= 6]

# 3. Remove players with 0 or NaN minutes played → considered inactive
player_statistics_df = player_statistics_df[player_statistics_df['numMinutes'].fillna(0) > 0]

# 4. Convert gameDate to datetime format
player_statistics_df['gameDate'] = pd.to_datetime(player_statistics_df['gameDate'])

# 5. Fix swapped made/attempted values and recalculate shooting percentages.

# Fix swapped field goals
mask_fg_swapped = player_statistics_df['fieldGoalsMade'] > player_statistics_df['fieldGoalsAttempted']
player_statistics_df.loc[mask_fg_swapped, ['fieldGoalsMade', 'fieldGoalsAttempted']] = (
    player_statistics_df.loc[mask_fg_swapped, ['fieldGoalsAttempted', 'fieldGoalsMade']].values
)

# Fix swapped free throws
mask_ft_swapped = player_statistics_df['freeThrowsMade'] > player_statistics_df['freeThrowsAttempted']
player_statistics_df.loc[mask_ft_swapped, ['freeThrowsMade', 'freeThrowsAttempted']] = (
    player_statistics_df.loc[mask_ft_swapped, ['freeThrowsAttempted', 'freeThrowsMade']].values
)

# Set percentage = 0 if attempted == 0
player_statistics_df['fieldGoalsPercentage'] = np.where(
    player_statistics_df['fieldGoalsAttempted'] > 0,
    player_statistics_df['fieldGoalsMade'] / player_statistics_df['fieldGoalsAttempted'],
    0.0  # instead of np.nan
)

player_statistics_df['freeThrowsPercentage'] = np.where(
    player_statistics_df['freeThrowsAttempted'] > 0,
    player_statistics_df['freeThrowsMade'] / player_statistics_df['freeThrowsAttempted'],
    0.0
)

rows_after_cleaning = len(player_statistics_df)

print(f"Rows after cleaning:  {rows_after_cleaning:,}")

Rows after cleaning:  1,357,805


In [19]:
print("Detailed column overview after cleaning:\n")
for col in player_statistics_df.columns:
    dtype = player_statistics_df[col].dtype
    missing = player_statistics_df[col].isnull().sum()

    if dtype == 'object':
        example_values = player_statistics_df[col].dropna().unique()[:3]  # Beispiele zeigen
        examples = f"Examples: {str(list(example_values))}"
        print(f"| {str(col):<30} | {str(dtype):>15} | {examples:<80} | Missing: {missing:>10} |")
    else:
        min_val = player_statistics_df[col].min()
        max_val = player_statistics_df[col].max()
        r = f"Range: {min_val} to {max_val}"
        print(f"| {str(col):<30} | {str(dtype):>15} | {r:<80} | Missing: {missing:>10} |")

Detailed column overview after cleaning:

| firstName                      |          object | Examples: ['Kevin', 'Al', 'Jrue']                                                | Missing:          0 |
| lastName                       |          object | Examples: ['Durant', 'Horford', 'Holiday']                                       | Missing:          0 |
| personId                       |           int64 | Range: 2 to 1962936495                                                           | Missing:          0 |
| gameId                         |           int64 | Range: 10300001 to 62400001                                                      | Missing:          0 |
| gameDate                       |  datetime64[ns] | Range: 1951-11-11 19:00:00 to 2025-03-26 22:00:00                                | Missing:          0 |
| playerteamCity                 |          object | Examples: ['Phoenix', 'Boston', 'Denver']                                        | Missing:          0 |
| playerte

Rows before cleaning: 1,621,084

Rows after cleaning: 1,357,805

Removed rows: 263,279 → approx. 16.2% of all rows removed

In [20]:
# print all unique values for gameType
print("Unique values for gameType:")
print(player_statistics_df['gameType'].unique())

Unique values for gameType:
['Regular Season' 'Preseason' 'Playoffs' 'Play-in Tournament' 'NBA Cup'
 'NBA Emirates Cup']


In [22]:
# divide data into the groups of the game type; 'Regular Season' 'Preseason' 'Playoffs' 'Play-in Tournament' 'NBA Cup' 'NBA Emirates Cup'
regular_season_df = player_statistics_df[player_statistics_df['gameType'] == 'Regular Season']
preseason_df = player_statistics_df[player_statistics_df['gameType'] == 'Preseason']
playoffs_df = player_statistics_df[player_statistics_df['gameType'] == 'Playoffs']
play_in_tournament_df = player_statistics_df[player_statistics_df['gameType'] == 'Play-in Tournament']
nba_cup_df = player_statistics_df[player_statistics_df['gameType'] == 'NBA Cup']
nba_emirates_cup_df = player_statistics_df[player_statistics_df['gameType'] == 'NBA Emirates Cup']

# print nr of entries for each of the groups (new df's)
print("Nr of entries for each of the groups (new df's):")
print("Regular Season:", len(regular_season_df))
print("Preseason:", len(preseason_df))
print("Playoffs:", len(playoffs_df))
print("Play-in Tournament:", len(play_in_tournament_df))
print("NBA Cup:", len(nba_cup_df))
print("NBA Emirates Cup:", len(nba_emirates_cup_df))

Nr of entries for each of the groups (new df's):
Regular Season: 1229128
Preseason: 45165
Playoffs: 81600
Play-in Tournament: 508
NBA Cup: 24
NBA Emirates Cup: 1380


The NBA Play-In Tournament was officially introduced in the 2020–21 season to determine the final two playoff spots in each conference. It was designed to increase late-season competitiveness, reduce tanking, and keep more teams in playoff contention. While it is part of the postseason structure, it is not officially considered part of the NBA Playoffs.

In [23]:
# Extract comparable information
print("\nNBA Cup – gameDates & Type")
print(nba_cup_df[['gameDate', 'gameType', 'gameSubLabel']].drop_duplicates().sort_values('gameDate').head(10))

print("\nNBA Emirates Cup – gameDates & Type")
print(nba_emirates_cup_df[['gameDate', 'gameType', 'gameSubLabel']].drop_duplicates().sort_values('gameDate').head(10))


NBA Cup – gameDates & Type
                 gameDate gameType gameSubLabel
57604 2023-12-09 20:30:00  NBA Cup          NaN

NBA Emirates Cup – gameDates & Type
                 gameDate          gameType gameSubLabel
64210 2023-11-03 19:00:00  NBA Emirates Cup          NaN
64181 2023-11-03 19:30:00  NBA Emirates Cup          NaN
64100 2023-11-03 20:00:00  NBA Emirates Cup          NaN
64047 2023-11-03 22:00:00  NBA Emirates Cup          NaN
62889 2023-11-10 19:00:00  NBA Emirates Cup          NaN
62865 2023-11-10 19:30:00  NBA Emirates Cup          NaN
62790 2023-11-10 20:00:00  NBA Emirates Cup          NaN
62764 2023-11-10 20:30:00  NBA Emirates Cup          NaN
62708 2023-11-10 22:00:00  NBA Emirates Cup          NaN
62128 2023-11-14 19:00:00  NBA Emirates Cup          NaN


The NBA introduced the In-Season Tournament in the 2023/24 season, which has been officially named the "NBA Emirates In-Season Tournament" due to sponsorship. According to the NBA, all group stage games as well as the quarterfinals and semifinals of the tournament are part of the Regular Season and their statistics are treated accordingly. However, the Championship Game (the final match between the last two teams) is considered an additional 83rd game and does not count towards the Regular Season. This includes both the game result and all associated player statistics.

In [24]:
# Select all games with the sublabel "Championship"
championship_games_df = player_statistics_df[player_statistics_df['gameSubLabel'] == 'Championship']

# Count and display overview
print(f"Number of Championship games found: {len(championship_games_df)}")
display(
    championship_games_df[
        ['gameDate', 'gameType', 'gameLabel', 'gameSubLabel']
    ].drop_duplicates().sort_values('gameDate')
)

Number of Championship games found: 25


,gameDate,gameType,gameLabel,gameSubLabel
18393,2024-12-17 20:30:00,Regular Season,Emirates NBA Cup,Championship


In [25]:
# Final Game 1: December 9, 2023 (likely 2023 IST Final)
final_game_2023 = player_statistics_df[player_statistics_df['gameDate'] == '2023-12-09 20:30:00']

# Final Game 2: December 17, 2024 (likely 2024 IST Final)
final_game_2024 = player_statistics_df[
    (player_statistics_df['gameDate'] == '2024-12-17 20:30:00') &
    (player_statistics_df['gameSubLabel'] == 'Championship')
]

# Show the unique team matchups
print("IST Final 2023 – Teams:")
display(final_game_2023[['playerteamName', 'opponentteamName']].drop_duplicates())

print("IST Final 2024 – Teams:")
display(final_game_2024[['playerteamName', 'opponentteamName']].drop_duplicates())

IST Final 2023 – Teams:


,playerteamName,opponentteamName
57604,Lakers,Pacers
57606,Pacers,Lakers


IST Final 2024 – Teams:


,playerteamName,opponentteamName
18393,Bucks,Thunder
18400,Thunder,Bucks


These two games — Lakers vs. Pacers on December 9, 2023, and Bucks vs. Thunder on December 17, 2024 — have been confirmed as the NBA In-Season Tournament Finals. As per NBA guidelines, they are not part of the regular season and will be excluded for the regular season dataset.

In [26]:
# All regular season games
regular_season_df = player_statistics_df[player_statistics_df['gameType'] == 'Regular Season']

# All Emirates Cup games excluding the Final
emirates_cup_non_final_df = player_statistics_df[
    (player_statistics_df['gameType'] == 'NBA Emirates Cup') &
    (player_statistics_df['gameSubLabel'] != 'Championship')
]

# Combine both
combined_df = pd.concat([regular_season_df, emirates_cup_non_final_df])

# Final cleanup: remove any remaining 'Championship' games (just to be sure)
regular_season_player_statistics_df = combined_df[
    combined_df['gameSubLabel'] != 'Championship'
].reset_index(drop=True)

In [27]:
# Basic structure
print("regular_season_player_statistics_df created")
print(f"Number of rows: {len(regular_season_player_statistics_df):,}")
print(f"Number of columns: {regular_season_player_statistics_df.shape[1]}")

# Preview first few rows
display(regular_season_player_statistics_df.head(5))

# Check if any 'Championship' games accidentally included
championship_check = regular_season_player_statistics_df[
    regular_season_player_statistics_df['gameSubLabel'] == 'Championship'
]
print(f"Games labeled 'Championship' still in dataset: {len(championship_check)}")

regular_season_player_statistics_df created
Number of rows: 1,230,483
Number of columns: 35


,firstName,lastName,personId,gameId,gameDate,playerteamCity,playerteamName,opponentteamCity,opponentteamName,gameType,...,threePointersPercentage,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints
0,Kevin,Durant,201142,22401058,2025-03-26 22:00:00,Phoenix,Suns,Boston,Celtics,Regular Season,...,0.750,5.0,5.0,1.0,1.0,0.0,1.0,2.0,1.0,-22.0
1,Al,Horford,201143,22401058,2025-03-26 22:00:00,Boston,Celtics,Phoenix,Suns,Regular Season,...,0.286,4.0,4.0,1.0,6.0,4.0,10.0,1.0,0.0,24.0
2,Jrue,Holiday,201950,22401058,2025-03-26 22:00:00,Boston,Celtics,Phoenix,Suns,Regular Season,...,0.400,2.0,2.0,1.0,2.0,0.0,2.0,1.0,0.0,28.0
3,Mason,Plumlee,203486,22401058,2025-03-26 22:00:00,Phoenix,Suns,Boston,Celtics,Regular Season,...,0.000,0.0,0.0,0.0,4.0,1.0,5.0,2.0,1.0,1.0
4,Vasilije,Micic,203995,22401058,2025-03-26 22:00:00,Phoenix,Suns,Boston,Celtics,Regular Season,...,0.000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,-3.0


Games labeled 'Championship' still in dataset: 0


In [28]:
print("Detailed column overview after cleaning:\n")
for col in regular_season_player_statistics_df.columns:
    dtype = regular_season_player_statistics_df[col].dtype
    missing = regular_season_player_statistics_df[col].isnull().sum()

    if dtype == 'object':
        example_values = regular_season_player_statistics_df[col].dropna().unique()[:3]
        examples = f"Examples: {str(list(example_values))}"
        print(f"| {str(col):<30} | {str(dtype):>15} | {examples:<80} | Missing: {missing:>10} |")
    else:
        min_val = regular_season_player_statistics_df[col].min()
        max_val = regular_season_player_statistics_df[col].max()
        r = f"Range: {min_val} to {max_val}"
        print(f"| {str(col):<30} | {str(dtype):>15} | {r:<80} | Missing: {missing:>10} |")

Detailed column overview after cleaning:

| firstName                      |          object | Examples: ['Kevin', 'Al', 'Jrue']                                                | Missing:          0 |
| lastName                       |          object | Examples: ['Durant', 'Horford', 'Holiday']                                       | Missing:          0 |
| personId                       |           int64 | Range: 2 to 1642530                                                              | Missing:          0 |
| gameId                         |           int64 | Range: 20000001 to 29901189                                                      | Missing:          0 |
| gameDate                       |  datetime64[ns] | Range: 1951-11-11 19:00:00 to 2025-03-26 22:00:00                                | Missing:          0 |
| playerteamCity                 |          object | Examples: ['Phoenix', 'Boston', 'Denver']                                        | Missing:          0 |
| playerte

gameLabel: Contains promotional event names (e.g., “Paris Game”), which are not analytically meaningful.

gameSubLabel: Mostly used to indicate In-Season Tournament rounds. Since only the Championship was excluded, this column is no longer needed.

seriesGameNumber: Always contains 0.0 in the regular season, providing no additional information.

In [29]:
# Drop columns that are no longer needed
cols_to_drop = ['gameLabel', 'gameSubLabel', 'seriesGameNumber']

regular_season_player_statistics_df = regular_season_player_statistics_df.sort_values(
    by='gameDate', ascending=False
).reset_index(drop=True)